In [1]:
import fitz
import os
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from PIL import Image 
import io
import boto3
import numpy as np

F:\anaconda3\envs\python_3_7\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')



In [2]:
from PIL import Image
import pytesseract
import argparse
import cv2
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import csv
import nltk
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet 

# Code for Converting PDF to IMG

In [49]:

def convert_pdf2img(input_file: str,pages=None):
    """Converts pdf to image and generates a file by page"""
    # Open the document
    pdfIn = fitz.open(input_file)
    output_files = []
    # Iterate throughout the pages
    for pg in range(pdfIn.pageCount):
        if str(pages) != str(None):
            if str(pg) not in str(pages):
                continue
        # Select a page
        page = pdfIn[pg]
        rotate = int(0)
        # PDF Page is converted into a whole picture 1056*816 and then for each picture a screenshot is taken.
        # zoom = 1.33333333 -----> Image size = 1056*816
        # zoom = 2 ---> 2 * Default Resolution (text is clear, image text is hard to read)    = filesize small / Image size = 1584*1224
        # zoom = 4 ---> 4 * Default Resolution (text is clear, image text is barely readable) = filesize large
        # zoom = 8 ---> 8 * Default Resolution (text is clear, image text is readable) = filesize large
        zoom_x = 2
        zoom_y = 2
        # The zoom factor is equal to 2 in order to make text clear
        # Pre-rotate is to rotate if needed.
        mat = fitz.Matrix(zoom_x, zoom_y).preRotate(rotate)
        pix = page.getPixmap(matrix=mat, alpha=False)
        path = 'Makro Invoices/Makro Invoices/'
        output_file = f"{os.path.splitext(os.path.basename(input_file))[0]}_page{pg+1}.png"
        pix.writePNG(output_file)
        output_files.append(output_file)
    pdfIn.close()
    summary = {
        "File": input_file, "Pages": str(pages), "Output File(s)": str(output_files)
    }
    # Printing Summary
    print("## Summary ########################################################")
    print("\n".join("{}:{}".format(i, j) for i, j in summary.items()))
    print("###################################################################")
    return output_files

# Code for Converting PDF to IMG

In [51]:
convert_pdf2img('Makro Invoices/Makro Invoices/61908_A28647451 DEFAULT C9627 TYPE_20220530_6_FacturaCompra.pdf')

## Summary ########################################################
File:Makro Invoices/Makro Invoices/61908_A28647451 DEFAULT C9627 TYPE_20220530_6_FacturaCompra.pdf
Pages:None
Output File(s):['61908_A28647451 DEFAULT C9627 TYPE_20220530_6_FacturaCompra_page1.png']
###################################################################


['61908_A28647451 DEFAULT C9627 TYPE_20220530_6_FacturaCompra_page1.png']

# Code For Extracting Fields using Tesseract 

In [3]:
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

In [29]:
%%time
dat = {"Supplier":'',
    "Vat Number":'',
       "Total":0,
       "Date":'',
      "Invoice_Number":''}
img  = cv2.imread('Makro Invoices/Makro Invoices/Makro.png', cv2.COLOR_RGB2GRAY)
text=(pytesseract.image_to_string(img))
text_list = text.split("\n")
print(type(text_list))
for idx, a in enumerate(text_list):
    if(a.strip() != ""):
        if("Montesano" in a):
            dat["Supplier"] = "Montesano"
        if("Makro" in a or "MAKRO" in a or "makro" in a):
            dat["Supplier"] = "Makro"
        if("NIF:" in a or  "N.I.F.:" in a  or "CIF" in a):
                print("Case Vat Number")
                b = a.split(" ")
                if len(b[-1])>10:
                    dat["Vat Number"] = b[-1]
                    print(b[-1])
                elif dat["Supplier"] == "Montesano" and len(b[-1])>8:
                    dat["Vat Number"] = b[-1]
        if ("Tarjeta" in a or "Credito" in a or  "pagina" in a or "Total:" in a):
            print("Case Total")
            b = a.split(" ")
            if len(b[-1])==2:
                
                dat["Total"] = b[-2]+b[-1]
                print(b[-2]+b[-1])
            else:
                dat["Total"] = b[-1]
                print(b[-1])
        if ("vent:" in a or "venta:" in a or "Fecha:" in a or "Fecha" in a):
            print("Date")
            b = a.split(" ")
            if len(b[-1])>7:
                dat["Date"] = b[-1]
                print(b[-1])
            else:
                dat["Date"] = b[-2]
                print(b[-2])
        if ("Factura" in a or "ALBARAN" in a or "Factura: " in a):

            print(a)
            b = a.split(" ")
            if b[0] == "ALBARAN":
                dat["Invoice_Number"] = b[1]
                print("Hello Darling")
            print(b)
            if len(b)>3:
                print(b)
                if len(b)>4 and b[2] !='—' and len(b[1])!=25 and len(b[1])>5:
                    dat["Invoice_Number"] = b[1]+b[2]
                    print(b)
                elif len(b)>=5 and len(b[1])==25:
                    dat["Invoice_Number"] = b[1]
                elif len(b)>6 and len(b[3])>15:
                    dat["Invoice_Number"] = b[1]+ b[2]+ b[3]
                elif len(b)>6 and b[1]== "0/0":
                    dat["Invoice_Number"] = b[1] + b[2]

<class 'list'>
Factura
['Factura']
Date
02/01/2021
Date
02.01.2021
Date
02/01/2022
Case Total
10,
Wall time: 4.15 s


In [30]:
dat

{'Supplier': 'Makro',
 'Vat Number': '',
 'Total': '10,',
 'Date': '02/01/2022',
 'Invoice_Number': ''}

# Code for Extracting Table 

In [31]:
im = Image.open('Makro Invoices/Makro Invoices/Makro.png')

In [32]:
buffered = io.BytesIO()
im.save(buffered, format='PNG')

In [33]:
client = boto3.client('textract')
response = client.analyze_document(
    Document={'Bytes': buffered.getvalue()},
    FeatureTypes=['TABLES']
)

In [34]:
def map_blocks(blocks, block_type):
    return {
        block['Id']: block
        for block in blocks
        if block['BlockType'] == block_type
    }

blocks = response['Blocks']
tables = map_blocks(blocks, 'TABLE')
cells = map_blocks(blocks, 'CELL')
words = map_blocks(blocks, 'WORD')
selections = map_blocks(blocks, 'SELECTION_ELEMENT')

In [35]:
def get_children_ids(block):
    for rels in block.get('Relationships', []):
        if rels['Type'] == 'CHILD':
            yield from rels['Ids']

In [36]:
import pandas as pd

dataframes = []

for table in tables.values():

    # Determine all the cells that belong to this table
    table_cells = [cells[cell_id] for cell_id in get_children_ids(table)]

    # Determine the table's number of rows and columns
    n_rows = max(cell['RowIndex'] for cell in table_cells)
    n_cols = max(cell['ColumnIndex'] for cell in table_cells)
    content = [[None for _ in range(n_cols)] for _ in range(n_rows)]

    # Fill in each cell
    for cell in table_cells:
        cell_contents = [
            words[child_id]['Text']
            if child_id in words
            else selections[child_id]['SelectionStatus']
            for child_id in get_children_ids(cell)
        ]
        i = cell['RowIndex'] - 1
        j = cell['ColumnIndex'] - 1
        content[i][j] = ' '.join(cell_contents)

    # We assume that the first row corresponds to the column names
    dataframe = pd.DataFrame(content[1:], columns=content[0])
    dataframes.append(dataframe)

In [45]:

if "MM Num. articulo" in dataframes[1] or "MM Num. artículo" in dataframes[1] or "REF." in dataframes[1] or "Num. articulo" in dataframes[1] or "Num. artículo" in dataframes[1]:
    data = pd.DataFrame(dataframes[1])
    data.to_excel('Makro Invoices/Makro Invoices/Saved_Datad.xlsx')
    print("Yes 1")
elif "MM Num. articulo" in dataframes[0] or "MM Num. artículo" in dataframes[0] or  "Num. articulo" in dataframes[0]:
    data = pd.DataFrame(dataframes[0])
    data.to_excel('Makro Invoices/Makro Invoices/Saved_Datad.xlsx')
    print("Yes 2")

elif "Bsdc SL" in dataframes[0] :
    data = pd.DataFrame(dataframes[1])
    data.to_excel('Makro Invoices/Makro Invoices/Saved_Datad.xlsx')
    
elif (len(dataframes)>=3 and ("MM Num. artículo" in dataframes[2] or "MM Num. artículo" in dataframes[2] 
                            or "MM Num. articulo" in dataframes[2])):
    data = pd.DataFrame(dataframes[2])
    data.to_excel('Makro Invoices/Makro Invoices/Saved_Datad.xlsx')
    print('Condition 2')
    
else:
    data = pd.DataFrame(dataframes[0])
    data.to_excel('Makro Invoices/Makro Invoices/Saved_Datad.xlsx')
    print("Yes")

Yes 1


In [46]:
dataframes[1]

,Num. artículo,Descrip. artículo,Cont,,Prec. Ud.,Cont P.,Precio,Cant.,Importe,Imp,Prec.Ud.*. DT,AOO,RCD
0,08437006156653,LIMON GRANEL CJ/KG,KG,,"1,090","2,020","2,20",1,"2,20",0,,,
1,08437009243169,CEBOLLA ROJA CJ/KG,KG,,"1,250","3,165","3,96",1,"3,96",0,,,
2,2054846052400,COL VERDE CAMPO CJ/KG,KG,,"0,890","5,240","4,66",1,"4,66",0,,,
3,M 110359,PAPAS KING EDWARDS MC so 25KG,SC,,"16,250",1,"16,25",1,"16,25",0,,,
4,8437005995079,PADRON DULCE MC BL 500G,BL,,"3,750",1,"3,75",2,"7,50",0,,,
5,5449000279217,COCACOLA BT 20CL ZERO,BT,,"0,650",24,"15,59",2,"31,18",1,,,
6,5449000279187,COCACOLA BT 20CL REGULAR,BT,,"0,650",24,"15,59",2,"31,18",1,,,
7,Número de bultos:,"10 Peso Total: 10, 425 KG",,Envases:,0,,,,Importe,,"96,93",,


In [47]:
data.shape

(8, 13)

In [20]:
if dat["Supplier"] == "Makro":
    big = data.columns[0]
    cont = data.columns[2]
    lot = data.columns[1]
    prec = data.columns[3]
    cant = data.columns[-5]
    imp = data.columns[-2]
    importe = data.columns[-3]
    third = data.columns[-4]
    da = data.loc[(data[big] != "Número de pedido") & (data[big] != "*** Número de pedido") & (data[big]!="*** Número de") & (data[big]!="*** Número")]
    da = da.loc[(da[big] != "Entregado a: TABERNA") & (da[big] !="Entregado a: TASCA") & (da[big]!="Entregado a: TABERNA OSUNA")
          & (da[big]!= "Entregado a: TABERNA osuna")& (da[big]!="Fin de número")]
    da = da.loc[da[cont] !="LOT:"]
    da = da.loc[da[cant] !="Total"]
    da = da.loc[da[cant]!="LA LAGUNA" ]
    da = da.replace('',np.nan)
    da = da.dropna(axis=0, thresh=3, how="any")
    limitPer = len(da) * .100
    da = da.dropna(thresh=limitPer, axis=1)
    da = da.replace(np.nan,'')
    da = da.applymap(lambda x: str(x.replace(',','.')))
    #da[lot] = da[lot]+' ' + da[cont]
    #da =da.drop(labels=[cont,prec], axis=1)
    #typee = {da.columns[2]:int}
    #da = da.astype(typee)
    da = da.astype({'Precio':float,'Importe':float,'Imp':int})
    one_tax = da.loc[(da['Imp'] == 1)]
    tax_one = 0
    for i in one_tax['Importe']:
        tax_one +=i
    tax_one = round(tax_one*0.03,2)
    two_tax =  da.loc[(da['Imp'] == 2)]
    tax_two = 0
    for i in two_tax['Importe']:
        tax_two +=i
    tax_two = round(tax_two*0.07,2)
    final_total = 0
    for i in da['Importe']:
        final_total +=i
    final_total = round(final_total)
    final_sum = round(final_total + tax_one + tax_two,2)
    print(final_sum)
    print(dat["Total"])
    if str(final_sum) == dat["Total"]:
        print("Both Are equal")
    
elif dat["Supplier"] =="Montesano":
    da = data
    da = da.applymap(lambda x: str(x.replace(',','.')))
    da = da.astype({'Neto':float,'Total':float})
    dat['Total'] = dat['Total'].replace(',','.')
    tot = 0
    for i in da["Total"]:
        print(i)
        tot +=i
        tot = round(tot,2)
    three_percent = da.loc[(da['IGIC'] == '3%')]
    three_var = 0 
    for i in three_percent["Total"]:
        three_var +=i
    three_var = round(three_var *0.03,2)
    tot = three_var + tot
    tot = str(tot)
    print(tot,dat["Total"])
    if tot == dat["Total"]:
        print("Both Values are equal")
    
    

111.29
24.87
17.14
154.05 154.05
Both Values are equal


In [149]:
# drop cont  Prec. Ud. 
da.columns

Index(['MM Num. articulo', 'Descrip. articulo', 'Cont P.', 'Precio', 'Cant.',
       'Importe', 'Imp', 'DT AOO'],
      dtype='object')

In [160]:
da.head(100)

,MM Num. articulo,Descrip. articulo,Cont,Prec. Ud.,Cont P.,Precio,Cant.,Importe,Imp,DT AOO
0,8437017304685,VINO TTO REMIREZ GANUZA RVA75C,BT,45.250,1,45.25,1,45.25,2,
1,8436557311009,VINO TINTO LA FELISA 75 CL,BT,21.150,1,21.15,6,126.90,2,c
2,8436014246486,VINO TTO VALBUENA 5 AN 75CL,BT,102.790,1,102.79,2,205.58,2,
3,8423339924014,VINO TTO PESQUERA CZA RD 75CL,BT,16.290,1,16.29,6,97.74,2,P
4,18414892358665,REFRESCO ARO PT 2L LIMON ASB,BO,0.615,6,3.69,1,3.69,1,
5,8414892700696,CAZUELA RDA BARRO MPRO 32CM,PZ,7.350,1,7.35,1,7.35,1,P
6,4337147823893,PLATO TAPAS CUADRADO MPR06CM,BS,0.850,6,5.10,2,10.20,1,
7,8428141244435,BOWL DOBLE MING 17X18X4CM,UD,3.500,1,3.50,9,31.50,1,
8,8414892816298,COPA VINO PRIME TIME ARO 52.5C,UD,0.920,1,0.92,12,11.04,1,


In [158]:
da.shape

(9, 10)

In [159]:
da.columns

Index(['MM Num. articulo', 'Descrip. articulo', 'Cont', 'Prec. Ud.', 'Cont P.',
       'Precio', 'Cant.', 'Importe', 'Imp', 'DT AOO'],
      dtype='object')

In [56]:
fi = db.astype({'NOT_SELECTED Precio':float,'Importe':float,'Imp Prec. Ud. DT':int})

In [61]:
tot = 0
for i in fi['Importe']:
    tot +=i
    tot = round(tot,2)
print(tot)

325.11


In [51]:
fi.dtypes

MM Num. articulo      object
Descrip. articulo     object
Cont                  object
Prec. Ud.            float64
Cont P.               object
Precio               float64
Cant.                  int32
Importe              float64
Imp                    int32
dtype: object

In [63]:
one_tax = fi.loc[(fi['Imp Prec. Ud. DT'] == 1)]

In [64]:
one_tax 
three_tax = 0
for i in one_tax['Importe']:
    three_tax +=i
three_tax = round(three_tax * 0.03,2)

print(three_tax)

6.94


In [65]:
two_tax = fi.loc[(fi['Imp Prec. Ud. DT'] == 2)]

In [66]:
two_tax
seven_tax = 0
for i in two_tax['Importe']:
    seven_tax +=i
seven_tax = round(seven_tax *0.07,2)
print(seven_tax)

2.52


In [67]:
final_sum = round(tot+ three_tax + seven_tax,2)
print(final_sum)

334.57


In [79]:
dat['Total'] = dat['Total'].replace(',','.')
#dat = dat.applymap(lambda x: str(x.replace(',','.')))
print(dat)

{'Vat Number': 'A-28/647451', 'Total': '334.57', 'Date': '20/05/202'}


In [81]:
if str(final_sum) == dat['Total']:
    print(final_sum,"Both Sums are equal",dat['Total'])

334.57 Both Sums are equal 334.57


In [82]:
fi.head(100)

,MM Num. artículo,Descrip. artículo,Cont,Prec. Ud.,Cont P.,NOT_SELECTED Precio,Cant.,Importe,Imp Prec. Ud. DT,AOO RC
2,0,0,UD,10.260,1,10.26,4,41.04,0,0
3,091867,Tello Codillo asado 700g,PZ,5.770,1,5.77,18,103.86,1,0
4,98435223249021,Atún lomo sin piel 2-4piezas/kg,FAO 51,Océano Índico,Oeste,0.00,0,0.00,0,0
5,0,0,KG,16.590,4.734,78.54,1,78.54,1,HD
6,0,GTIN: 98435223249021 Lote: 5 GTIN: 98435223249...,LOT: 5,0,0,0.00,0,0.00,0,0
7,078438,makro Chef Manjares de mar 2x200g,BJ,2.850,1,2.85,5,14.25,1,0
8,155329,Altamar Almeja marron congelada,60/80,piezas 1kg,0,0.00,0,0.00,0,0
9,0,0,BL,3.650,1,3.65,2,7.30,0,0
10,111089,Signo Arroz bomba bolsa 1 kg,PQ,2.950,1,2.95,3,8.85,1,0
11,451555,makro Chef Mayonesa cubo 51,CB,17.300,1,17.30,1,17.30,1,0
